In [1]:
import pandas as pd
import contractions
import re
import nltk
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/alvinrach/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-08-21 21:29:19.703811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-21 21:29:19.716698: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-21 21:29:19.720771: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-21 21:29:19.731352: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructi

In [2]:
d = pd.read_csv('https://raw.githubusercontent.com/alvinrach/learn-ai-bbc/main/BBC%20News%20Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/alvinrach/learn-ai-bbc/main/BBC%20News%20Test.csv')
sample = pd.read_csv('https://raw.githubusercontent.com/alvinrach/learn-ai-bbc/main/BBC%20News%20Sample%20Solution.csv')

In [3]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ArticleId  1490 non-null   int64 
 1   Text       1490 non-null   object
 2   Category   1490 non-null   object
dtypes: int64(1), object(2)
memory usage: 35.1+ KB


In [4]:
d = d.drop('ArticleId',axis=1)

In [5]:
d

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business
...,...,...
1485,double eviction from big brother model caprice...,entertainment
1486,dj double act revamp chart show dj duo jk and ...,entertainment
1487,weak dollar hits reuters revenues at media gro...,business
1488,apple ipod family expands market apple has exp...,tech


In [6]:
def txtprocess(txt):
    txt = str(txt).lower()
    txt = contractions.fix(txt)

    txt = re.sub(r'[^a-zA-Z]', ' ', txt)
    txt = re.sub(' +', ' ', txt)

    txt = ' '.join(txt.split())

    return txt

d['Text'] = d['Text'].apply(txtprocess)

In [7]:
stop_words = set(nltk.corpus.stopwords.words('english'))

# kayak you'll gitu masih ada ' nya , apa bagusnya sebelum txtprocess, tapi kecil semua sih
def remove_stopwords(txt):
    no_stopword_txt = [w for w in txt.split() if not w in stop_words]
    return ' '.join(no_stopword_txt)

d['Text'] = d['Text'].apply(lambda x: remove_stopwords(x))

In [8]:
# Alternative but just not

In [ ]:
def remove_stopwords(txt):
    no_stopword_txt = [w for w in txt.split() if not w in stop_words]
    return ' '.join(no_stopword_txt)

def txtprocess(txt):
    txt = str(txt).lower()
    txt = contractions.fix(txt)
    txt = remove_stopwords(txt)

    txt = re.sub(r'[^a-zA-Z]', ' ', txt)
    txt = re.sub(' +', ' ', txt)

    txt = ' '.join(txt.split())

    return txt

d['Text'] = d['Text'].apply(txtprocess)

In [9]:
category = pd.get_dummies(d.Category, dtype=int)
d_new = pd.concat([d, category], axis=1)
d_new = d_new.drop('Category', axis=1)
d_new

,Text,business,entertainment,politics,sport,tech
0,worldcom ex boss launches defence lawyers defe...,1,0,0,0,0
1,german business confidence slides german busin...,1,0,0,0,0
2,bbc poll indicates economic gloom citizens maj...,1,0,0,0,0
3,lifestyle governs mobile choice faster better ...,0,0,0,0,1
4,enron bosses payout eighteen former enron dire...,1,0,0,0,0
...,...,...,...,...,...,...
1485,double eviction big brother model caprice holb...,0,1,0,0,0
1486,dj double act revamp chart show dj duo jk joel...,0,1,0,0,0
1487,weak dollar hits reuters revenues media group ...,1,0,0,0,0
1488,apple ipod family expands market apple expande...,0,0,0,0,1


In [10]:
article = d_new['Text'].values
label = d_new[category.columns].values

In [11]:
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Tokenize directly (no fit_on_texts needed - it's already pretrained!)
tokens = tokenizer(article.tolist(), padding=True, truncation=True, return_tensors='pt')

# Split the tokenized data
padded_train, padded_test, y_train, y_test = train_test_split(
    tokens['input_ids'], label, test_size=0.2, random_state=42
)

/home/alvinrach/.jupytervenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
padded_train, y_train

(tensor([[  101, 11865,  6562,  ...,     0,     0,     0],
         [  101,  4121,  5481,  ...,     0,     0,     0],
         [  101,  7206,  3404,  ...,     0,     0,     0],
         ...,
         [  101, 23413,  2229,  ...,     0,     0,     0],
         [  101,  3153,  2189,  ...,     0,     0,     0],
         [  101,  3306,  3940,  ...,     0,     0,     0]]),
 array([[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        ...,
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0]]))

In [13]:
tokenizer.vocab_size

30522

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size+1, output_dim=500, mask_zero=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Purist, kureng. masih bingung sebenarnya padding_idx=1 itu masked apa engga
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=500, hidden_dim=64, output_dim=5):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size+1, embedding_dim=embedding_dim, padding_idx=1)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        output, (h_n, c_n) = self.lstm(x)  # output: [batch_size, seq_len, hidden_dim]
        x = h_n[-1]  # take the last hidden state
        x = self.fc(x)  # [batch_size, output_dim]
        x = F.softmax(x, dim=1)
        return x

# Example usage
vocab_size = tokenizer.vocab_size
model = MyModel(vocab_size)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # same as categorical_crossentropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

In [ ]:
# dengan pad packed sequence, tp padding_idx 0
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=500, hidden_dim=64, output_dim=5):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size+1, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: [batch_size, seq_len]
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]

        # compute lengths of non-padded tokens
        lengths = torch.sum(x.abs().sum(dim=2) != 0, dim=1)  # or use original input: x_input != 0
        # pack the sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        # pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed)
        # unpack if needed (not necessary if just taking last hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        # take last hidden state for classification
        x = h_n[-1]  # [batch_size, hidden_dim]
        x = self.fc(x)  # [batch_size, output_dim]
        x = F.softmax(x, dim=1)
        return x

# Example usage
vocab_size = tokenizer.vocab_size
model = MyModel(vocab_size)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # same as categorical_crossentropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

In [20]:
# katanya sih biar mimic keras kita harus mimic weight nya keras dan pake forget gate bias nya keras
# ini yang terbaik
# Epoch 46/60, Train Loss: 0.9059, Val Loss: 0.9397, Val Acc: 0.9698
# Epoch 47/60, Train Loss: 0.9059, Val Loss: 0.9399, Val Acc: 0.9732
# Epoch 48/60, Train Loss: 0.9059, Val Loss: 0.9408, Val Acc: 0.9664
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=500, hidden_dim=64, output_dim=5):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size+1, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: [batch_size, seq_len]
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]

        # compute lengths of non-padded tokens
        lengths = torch.sum(x.abs().sum(dim=2) != 0, dim=1)  # or use original input: x_input != 0
        # pack the sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        # pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed)
        # unpack if needed (not necessary if just taking last hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        # take last hidden state for classification
        x = h_n[-1]  # [batch_size, hidden_dim]
        x = self.fc(x)  # [batch_size, output_dim]
        x = F.softmax(x, dim=1)
        return x

# Example usage
vocab_size = tokenizer.vocab_size
model = MyModel(vocab_size)

# 4) Weight initialization to mimic Keras
def init_lstm_like_keras(m):
    if isinstance(m, nn.Embedding):
        # Keras embed init is usually uniform small; this is OK
        nn.init.normal_(m.weight, mean=0.0, std=0.01)
        if m.padding_idx is not None:
            with torch.no_grad():
                m.weight[m.padding_idx].fill_(0)
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    if isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)   # kernel ~ glorot
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)       # recurrent ~ orthogonal
            elif 'bias' in name:
                nn.init.zeros_(param.data)

model.apply(init_lstm_like_keras)

# 5) Set forget-gate bias = 1 (handles bias_ih + bias_hh)
for names in model.lstm._all_weights:
    for name in names:
        if 'bias' in name:
            bias = getattr(model.lstm, name)
            n = bias.size(0)
            # gates are i, f, g, o => forget gate slice is n//4 to n//4*2
            start = n // 4
            end = start + n // 4
            with torch.no_grad():
                bias[start:end].fill_(1.0)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # same as categorical_crossentropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

In [ ]:
# sebenarnya perlu ga sih pad packed sequence
# ternyata malah makin jelek. jadi exp ke 3 diambil
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=500, hidden_dim=64, output_dim=5):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size+1, embedding_dim=embedding_dim, padding_idx=1)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]
        output, (h_n, c_n) = self.lstm(x)  # output: [batch_size, seq_len, hidden_dim]
        x = h_n[-1]  # take the last hidden state
        x = self.fc(x)  # [batch_size, output_dim]
        x = F.softmax(x, dim=1)
        return x

# Example usage
vocab_size = tokenizer.vocab_size
model = MyModel(vocab_size)

# 4) Weight initialization to mimic Keras
def init_lstm_like_keras(m):
    if isinstance(m, nn.Embedding):
        # Keras embed init is usually uniform small; this is OK
        nn.init.normal_(m.weight, mean=0.0, std=0.01)
        if m.padding_idx is not None:
            with torch.no_grad():
                m.weight[m.padding_idx].fill_(0)
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    if isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)   # kernel ~ glorot
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)       # recurrent ~ orthogonal
            elif 'bias' in name:
                nn.init.zeros_(param.data)

model.apply(init_lstm_like_keras)

# 5) Set forget-gate bias = 1 (handles bias_ih + bias_hh)
for names in model.lstm._all_weights:
    for name in names:
        if 'bias' in name:
            bias = getattr(model.lstm, name)
            n = bias.size(0)
            # gates are i, f, g, o => forget gate slice is n//4 to n//4*2
            start = n // 4
            end = start + n // 4
            with torch.no_grad():
                bias[start:end].fill_(1.0)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # same as categorical_crossentropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

In [16]:
# katanya sih biar mimic keras kita harus mimic weight nya keras dan pake forget gate bias nya keras
# ini yang terbaik
# Tapi apakah karena weight?
# Epoch 59/60, Train Loss: 0.9058, Val Loss: 1.0580, Val Acc: 0.8389
# Epoch 60/60, Train Loss: 0.9058, Val Loss: 1.0588, Val Acc: 0.8423
# Good but stuck
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=500, hidden_dim=64, output_dim=5):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size+1, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: [batch_size, seq_len]
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]

        # compute lengths of non-padded tokens
        lengths = torch.sum(x.abs().sum(dim=2) != 0, dim=1)  # or use original input: x_input != 0
        # pack the sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        # pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed)
        # unpack if needed (not necessary if just taking last hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        # take last hidden state for classification
        x = h_n[-1]  # [batch_size, hidden_dim]
        x = self.fc(x)  # [batch_size, output_dim]
        x = F.softmax(x, dim=1)
        return x

# Example usage
vocab_size = tokenizer.vocab_size
model = MyModel(vocab_size)

# 4) Weight initialization to mimic Keras
def init_lstm_like_keras(m):
    if isinstance(m, nn.Embedding):
        # Keras embed init is usually uniform small; this is OK
        nn.init.normal_(m.weight, mean=0.0, std=0.01)
        if m.padding_idx is not None:
            with torch.no_grad():
                m.weight[m.padding_idx].fill_(0)
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    if isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)   # kernel ~ glorot
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)       # recurrent ~ orthogonal
            elif 'bias' in name:
                nn.init.zeros_(param.data)

model.apply(init_lstm_like_keras)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # same as categorical_crossentropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

In [22]:
# katanya sih biar mimic keras kita harus mimic weight nya keras dan pake forget gate bias nya keras
# ini yang terbaik
# Atau karena forget gate bias aja?
# Epoch 49/60, Train Loss: 0.9062, Val Loss: 0.9934, Val Acc: 0.9128
# Epoch 50/60, Train Loss: 0.9062, Val Loss: 0.9933, Val Acc: 0.9128
# Epoch 51/60, Train Loss: 0.9061, Val Loss: 0.9933, Val Acc: 0.9094
# Epoch 52/60, Train Loss: 0.9061, Val Loss: 0.9936, Val Acc: 0.9094
# Mmg mostly karena forget gate bias. Tapi dia ga sampe 97% kek yg exp ketiga
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class MyModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=500, hidden_dim=64, output_dim=5):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size+1, embedding_dim=embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x: [batch_size, seq_len]
        x = self.embedding(x)  # [batch_size, seq_len, embedding_dim]

        # compute lengths of non-padded tokens
        lengths = torch.sum(x.abs().sum(dim=2) != 0, dim=1)  # or use original input: x_input != 0
        # pack the sequence
        packed = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        # pass through LSTM
        packed_output, (h_n, c_n) = self.lstm(packed)
        # unpack if needed (not necessary if just taking last hidden)
        output, _ = pad_packed_sequence(packed_output, batch_first=True)

        # take last hidden state for classification
        x = h_n[-1]  # [batch_size, hidden_dim]
        x = self.fc(x)  # [batch_size, output_dim]
        x = F.softmax(x, dim=1)
        return x

# Example usage
vocab_size = tokenizer.vocab_size
model = MyModel(vocab_size)

# 5) Set forget-gate bias = 1 (handles bias_ih + bias_hh)
for names in model.lstm._all_weights:
    for name in names:
        if 'bias' in name:
            bias = getattr(model.lstm, name)
            n = bias.size(0)
            # gates are i, f, g, o => forget gate slice is n//4 to n//4*2
            start = n // 4
            end = start + n // 4
            with torch.no_grad():
                bias[start:end].fill_(1.0)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # same as categorical_crossentropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, eps=1e-7)

In [ ]:
#cpu
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Convert your data to PyTorch tensors
X_train_tensor = padded_train.clone().detach().long()
y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long)  # if one-hot
X_val_tensor = padded_test.clone().detach().long()
y_val_tensor = torch.tensor(np.argmax(y_test, axis=1), dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

# Early stopping parameters
patience = 15
best_val_loss = float('inf')
counter = 0

num_epochs = 60

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item() * X_batch.size(0)
            correct += (y_pred.argmax(1) == y_batch).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        best_model_state = model.state_dict()  # save best model
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            model.load_state_dict(best_model_state)  # restore best model
            break

/tmp/ipython-input-3726492126.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(padded_train, dtype=torch.long)
/tmp/ipython-input-3726492126.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(padded_test, dtype=torch.long)


Epoch 1/60, Train Loss: 1.6067, Val Loss: 1.6081, Val Acc: 0.2181
Epoch 2/60, Train Loss: 1.6019, Val Loss: 1.6062, Val Acc: 0.2181
Epoch 3/60, Train Loss: 1.5980, Val Loss: 1.6046, Val Acc: 0.2181


KeyboardInterrupt: 

In [15]:
#gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move model to GPU
model = model.to(device)


import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Convert your data to PyTorch tensors
X_train_tensor = padded_train.clone().detach().long()
y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long)  # if one-hot
X_val_tensor = padded_test.clone().detach().long()
y_val_tensor = torch.tensor(np.argmax(y_test, axis=1), dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)

# Early stopping parameters
patience = 15
best_val_loss = float('inf')
counter = 0

num_epochs = 60

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            val_loss += loss.item() * X_batch.size(0)
            correct += (y_pred.argmax(1) == y_batch).sum().item()
    val_loss /= len(val_loader.dataset)
    val_acc = correct / len(val_loader.dataset)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        best_model_state = model.state_dict()  # save best model
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered")
            model.load_state_dict(best_model_state)  # restore best model
            break

Using device: cuda


/tmp/ipykernel_6681/35588772.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(padded_train, dtype=torch.long)
/tmp/ipykernel_6681/35588772.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_tensor = torch.tensor(padded_test, dtype=torch.long)


Epoch 1/60, Train Loss: 1.6056, Val Loss: 1.5985, Val Acc: 0.2617
Epoch 2/60, Train Loss: 1.5993, Val Loss: 1.5976, Val Acc: 0.2617
Epoch 3/60, Train Loss: 1.5972, Val Loss: 1.6010, Val Acc: 0.2181
Epoch 4/60, Train Loss: 1.5949, Val Loss: 1.6023, Val Acc: 0.2181
Epoch 5/60, Train Loss: 1.5927, Val Loss: 1.6013, Val Acc: 0.2181
Epoch 6/60, Train Loss: 1.5901, Val Loss: 1.6016, Val Acc: 0.2148
Epoch 7/60, Train Loss: 1.5879, Val Loss: 1.6007, Val Acc: 0.2148
Epoch 8/60, Train Loss: 1.5864, Val Loss: 1.6001, Val Acc: 0.2148
Epoch 9/60, Train Loss: 1.5849, Val Loss: 1.6021, Val Acc: 0.2148
Epoch 10/60, Train Loss: 1.5835, Val Loss: 1.6010, Val Acc: 0.2148
Epoch 11/60, Train Loss: 1.5824, Val Loss: 1.6010, Val Acc: 0.2148
Epoch 12/60, Train Loss: 1.5818, Val Loss: 1.6007, Val Acc: 0.2148
Epoch 13/60, Train Loss: 1.5818, Val Loss: 1.5992, Val Acc: 0.2248
Epoch 14/60, Train Loss: 1.5815, Val Loss: 1.5988, Val Acc: 0.2685
Epoch 15/60, Train Loss: 1.5791, Val Loss: 1.5988, Val Acc: 0.2852
Epoc

In [ ]:
set_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15)

history = model.fit(padded_train, y_train, epochs=60, batch_size=128,
                   validation_data=(padded_test, y_test), callbacks=[set_callback], verbose=2)

Epoch 1/60
10/10 - 9s - 869ms/step - accuracy: 0.2718 - loss: 1.5916 - val_accuracy: 0.3221 - val_loss: 1.5562
Epoch 2/60
10/10 - 0s - 46ms/step - accuracy: 0.6611 - loss: 1.3697 - val_accuracy: 0.5906 - val_loss: 1.1325
Epoch 3/60
10/10 - 0s - 47ms/step - accuracy: 0.9111 - loss: 0.7094 - val_accuracy: 0.8389 - val_loss: 0.6645
Epoch 4/60
10/10 - 1s - 62ms/step - accuracy: 0.9874 - loss: 0.2542 - val_accuracy: 0.9262 - val_loss: 0.3035
Epoch 5/60
10/10 - 0s - 46ms/step - accuracy: 0.9975 - loss: 0.0524 - val_accuracy: 0.9329 - val_loss: 0.2563
Epoch 6/60
10/10 - 1s - 50ms/step - accuracy: 0.9983 - loss: 0.0177 - val_accuracy: 0.9396 - val_loss: 0.2441
Epoch 7/60
10/10 - 1s - 53ms/step - accuracy: 0.9992 - loss: 0.0094 - val_accuracy: 0.9362 - val_loss: 0.2699
Epoch 8/60
10/10 - 1s - 64ms/step - accuracy: 0.9992 - loss: 0.0088 - val_accuracy: 0.9295 - val_loss: 0.3358
Epoch 9/60
10/10 - 1s - 60ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accuracy: 0.9161 - val_loss: 0.3769
Epoch 10/

In [ ]:
model.evaluate(padded_test, y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9234 - loss: 0.3850


[0.39577338099479675, 0.9295302033424377]

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=tokenizer.vocab_size+1, output_dim=500, mask_zero=True),
    tf.keras.layers.LSTM(16, dropout=0.9),
    tf.keras.layers.Dense(5,activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_all = np.concatenate((padded_train, padded_test))
y_all = np.concatenate((y_train, y_test))

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<=history.history['loss'][-1]):
            print('Stopped, reach same losses as splitted trial')
            self.model.stop_training = True
set_callback = myCallback()

history = model.fit(X_all, y_all, epochs=60, batch_size=128,
                    callbacks=[set_callback], verbose=2)

Epoch 1/60
12/12 - 4s - 361ms/step - accuracy: 0.2315 - loss: 1.6064
Epoch 2/60
12/12 - 2s - 135ms/step - accuracy: 0.3107 - loss: 1.5851
Epoch 3/60
12/12 - 0s - 37ms/step - accuracy: 0.4054 - loss: 1.5488
Epoch 4/60
12/12 - 0s - 38ms/step - accuracy: 0.5094 - loss: 1.4383
Epoch 5/60
12/12 - 0s - 37ms/step - accuracy: 0.5174 - loss: 1.1650
Epoch 6/60
12/12 - 0s - 38ms/step - accuracy: 0.5591 - loss: 0.9750
Epoch 7/60
12/12 - 1s - 52ms/step - accuracy: 0.7584 - loss: 0.8106
Epoch 8/60
12/12 - 0s - 38ms/step - accuracy: 0.8517 - loss: 0.6544
Epoch 9/60
12/12 - 1s - 51ms/step - accuracy: 0.9081 - loss: 0.5152
Epoch 10/60
12/12 - 1s - 51ms/step - accuracy: 0.9430 - loss: 0.3807
Epoch 11/60
12/12 - 1s - 52ms/step - accuracy: 0.9658 - loss: 0.2818
Epoch 12/60
12/12 - 0s - 38ms/step - accuracy: 0.9732 - loss: 0.2333
Epoch 13/60
12/12 - 0s - 37ms/step - accuracy: 0.9839 - loss: 0.1710
Epoch 14/60
12/12 - 0s - 38ms/step - accuracy: 0.9839 - loss: 0.1545
Epoch 15/60
12/12 - 0s - 38ms/step - accu

In [ ]:
sample

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,business
3,459,entertainment
4,1020,politics
...,...,...
730,1923,sport
731,373,tech
732,1704,business
733,206,entertainment


In [ ]:
test['Text'] = test['Text'].apply(txtprocess)
test['Text'] = test['Text'].apply(lambda x: remove_stopwords(x))

testtext = test['Text'].values

paddedtesttext = tokenizer(testtext.tolist(), padding=True, truncation=True, return_tensors='pt')['input_ids']

In [ ]:
# 3️⃣ Move model and data to device
model.eval()  # important: set to evaluation mode
paddedtesttext = paddedtesttext.to(device)

# 4️⃣ Forward pass & get predictions
with torch.no_grad():
    outputs = model(paddedtesttext)  # [batch_size, output_dim]
    preds = torch.argmax(outputs, dim=1).cpu().numpy()  # convert to numpy

# 5️⃣ Map integer predictions to original category names
mapping = dict(enumerate(category.columns))
pred_labels = pd.Series(preds).map(mapping)

# 6️⃣ Combine with ArticleId
answer = pd.concat([test['ArticleId'].reset_index(drop=True), pred_labels.reset_index(drop=True)], axis=1)
answer.columns = ['ArticleId', 'Category']

answer

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,sport
3,459,business
4,1020,sport
...,...,...
730,1923,politics
731,373,entertainment
732,1704,politics
733,206,business


In [ ]:
torch.cuda.empty_cache()

In [ ]:
test_dataset = TensorDataset(paddedtesttext)
test_loader = DataLoader(test_dataset, batch_size=256)

all_preds = []
model.eval()
with torch.no_grad():
    for (X_batch,) in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)

# Map integer predictions to original category names
mapping = dict(enumerate(category.columns))
pred_labels = pd.Series(all_preds).map(mapping)

# Combine with ArticleId
answer = pd.concat([test['ArticleId'].reset_index(drop=True), pred_labels.reset_index(drop=True)], axis=1)
answer.columns = ['ArticleId', 'Category']

answer